In [1]:
import pandas as pd
import numpy as np

In [4]:
test = pd.read_csv("data/test.csv")
weather_test = pd.read_csv("data/weather_test.csv")
building_data = pd.read_csv("data/building_metadata.csv")

# Working on the test data

### 1. Merging all the three test data sets: original_test, building_data, weather_test

In [5]:
# add building data to the test data
test = pd.merge(test, building_data, on="building_id", how="inner")

# Now merge the data with the weather data
test = pd.merge(test, weather_test, on=["site_id", "timestamp"], how="left")

# Transforming data and filling na values

Suppose you have to fill temperature data for a nan temperature value. The best match for this nan value would be the data with the same site_id recorded at the same time. Luckily, we do not have any nan value for site_id or timestamp.

In [6]:
# sorting_by_site_datetime:
# 1. change the type of datetime from object to 'datetime'
test.timestamp = pd.to_datetime(test.timestamp, format='%Y-%m-%d %H:%M:%S')

# 2. sort w.r.t the site_id and then according to the timestamp for the same site_id
test.sort_values(inplace=True, by=['site_id', 'timestamp'])

In [7]:
test.isna().sum()

row_id                       0
building_id                  0
meter                        0
timestamp                    0
site_id                      0
primary_use                  0
square_feet                  0
year_built            24598080
floor_count           34444320
air_temperature         221901
cloud_coverage        19542180
dew_temperature         260799
precip_depth_1_hr      7801563
sea_level_pressure     2516826
wind_direction         2978663
wind_speed              302089
dtype: int64

In [8]:
prev_values=[]
columns=[]

prev_values.append(test.air_temperature.median())
columns.append("air_temperature")

prev_values.append(test.cloud_coverage.median())
columns.append("cloud_coverage")

prev_values.append(test.dew_temperature.median())
columns.append("dew_temperature")

prev_values.append(test.precip_depth_1_hr.median())
columns.append("precip_depth_1_hr")

prev_values.append(test.sea_level_pressure.median())
columns.append("sea_level_pressure")

prev_values.append(test.wind_direction.median())
columns.append("wind_direction")

prev_values.append(test.wind_speed.median())
columns.append("wind_speed")

In [9]:
for row in test.itertuples():
    row_ind = row.Index
    for i in range(len(columns)):
        if (str(test.at[row_ind, columns[i]])=='nan'):
            test.at[row_ind, columns[i]] = prev_values[i]
        else:
            prev_values[i] = test.at[row_ind, columns[i]]

In [10]:
# Fill year_built and floor_count with median values
floor_count_median = test.floor_count.median()
year_built_median = test.year_built.median()
test.floor_count.fillna(floor_count_median, inplace=True)
test.year_built.fillna(year_built_median, inplace=True)

In [11]:
test.isna().sum()

row_id                0
building_id           0
meter                 0
timestamp             0
site_id               0
primary_use           0
square_feet           0
year_built            0
floor_count           0
air_temperature       0
cloud_coverage        0
dew_temperature       0
precip_depth_1_hr     0
sea_level_pressure    0
wind_direction        0
wind_speed            0
dtype: int64

# Now let us work with the train data

In [12]:
train = pd.read_csv("data/train.csv")

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,2016-01-01 00:00:00,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
1,0,2016-01-01 01:00:00,24.4,NaN,21.1,-1.0,1020.2,70.0,1.5
2,0,2016-01-01 02:00:00,22.8,2.0,21.1,0.0,1020.2,0.0,0.0
3,0,2016-01-01 03:00:00,21.1,2.0,20.6,0.0,1020.1,0.0,0.0
4,0,2016-01-01 04:00:00,20.0,2.0,20.0,-1.0,1020.0,250.0,2.6
...,...,...,...,...,...,...,...,...,...
139768,15,2016-12-31 19:00:00,3.0,NaN,-8.0,NaN,NaN,180.0,5.7
139769,15,2016-12-31 20:00:00,2.8,2.0,-8.9,NaN,1007.4,180.0,7.7
139770,15,2016-12-31 21:00:00,2.8,NaN,-7.2,NaN,1007.5,180.0,5.1
139771,15,2016-12-31 22:00:00,2.2,NaN,-6.7,NaN,1008.0,170.0,4.6


In [13]:
weather_train = pd.read_csv("data/weather_train.csv")

## Merge all the three training data sets

In [16]:
# As mentioned in the kaggle Description, meter_reading for meter-0 is in kBTU and we need to change it to kWh.
train.meter_reading[train.meter==0]*=0.293

# Now merge add the building details to training data
train = pd.merge(train, building_data, on="building_id", how="inner")

# Now add the weather data to the above transformed data
train = pd.merge(train, weather_train, on=["site_id", "timestamp"], how="left")

C:\Users\piyus\AppData\Local\Temp\ipykernel_1920\2958664114.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.meter_reading[train.meter==0]*=0.293


In [18]:
train.isna().sum()

building_id                  0
meter                        0
timestamp                    0
meter_reading                0
site_id                      0
primary_use                  0
square_feet                  0
year_built            12127645
floor_count           16709167
air_temperature          96658
cloud_coverage         8825365
dew_temperature         100140
precip_depth_1_hr      3749023
sea_level_pressure     1231669
wind_direction         1449048
wind_speed              143676
dtype: int64

In [19]:
train.dropna(inplace=True)
train.shape

(321728, 16)

In [20]:
train.isna().sum()

building_id           0
meter                 0
timestamp             0
meter_reading         0
site_id               0
primary_use           0
square_feet           0
year_built            0
floor_count           0
air_temperature       0
cloud_coverage        0
dew_temperature       0
precip_depth_1_hr     0
sea_level_pressure    0
wind_direction        0
wind_speed            0
dtype: int64

In [24]:
train.corr()

C:\Users\piyus\AppData\Local\Temp\ipykernel_1920\2189804198.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  train.corr()


,building_id,meter,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
building_id,1.000000,NaN,0.208358,NaN,-0.141665,0.195173,0.007697,0.187297,-0.000437,0.001996,-0.000535,0.000322,0.001449,-0.000150,0.000373
meter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
meter_reading,0.208358,NaN,1.000000,NaN,-0.156957,0.612993,0.372491,0.626113,-0.015394,-0.011821,0.001669,0.001262,0.007768,-0.030353,-0.035744
site_id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
primary_use,-0.141665,NaN,-0.156957,NaN,1.000000,0.004772,0.162426,-0.234333,-0.006727,-0.001853,-0.006549,0.000143,0.007836,-0.006985,-0.006488
square_feet,0.195173,NaN,0.612993,NaN,0.004772,1.000000,0.265812,0.642057,-0.006530,0.004031,-0.007716,0.000391,0.008282,-0.003387,-0.002694
year_built,0.007697,NaN,0.372491,NaN,0.162426,0.265812,1.000000,0.291919,0.005804,-0.003525,0.007177,-0.000414,-0.008343,0.004841,0.004464
floor_count,0.187297,NaN,0.626113,NaN,-0.234333,0.642057,0.291919,1.000000,-0.001307,0.006665,-0.002237,0.000650,0.003118,0.002210,0.003141
air_temperature,-0.000437,NaN,-0.015394,NaN,-0.006727,-0.006530,0.005804,-0.001307,1.000000,0.080004,0.656597,-0.002843,-0.491020,0.498709,0.471196
cloud_coverage,0.001996,NaN,-0.011821,NaN,-0.001853,0.004031,-0.003525,0.006665,0.080004,1.000000,0.103219,0.015431,0.018659,0.155951,0.210514


In [22]:
# convert primary_use to categorical
train.primary_use = train.primary_use.astype("category").cat.codes

In [28]:
train.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,...,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,Year,Month,Week,Day,hour
6485009,565,0,2016-01-01 01:00:00,2.49050,4,0,15326,1954.0,2.0,9.4,...,-2.2,0.0,1021.4,360.0,3.1,2016,1,53,1,1
6485010,565,0,2016-01-01 02:00:00,2.12425,4,0,15326,1954.0,2.0,8.3,...,-2.8,0.0,1021.6,0.0,0.0,2016,1,53,1,2
6485011,565,0,2016-01-01 03:00:00,2.19750,4,0,15326,1954.0,2.0,7.8,...,-2.8,0.0,1021.7,90.0,1.5,2016,1,53,1,3
6485012,565,0,2016-01-01 04:00:00,2.34400,4,0,15326,1954.0,2.0,5.0,...,-0.6,0.0,1021.8,0.0,0.0,2016,1,53,1,4
6485013,565,0,2016-01-01 05:00:00,2.05100,4,0,15326,1954.0,2.0,5.6,...,-1.1,0.0,1021.6,130.0,3.1,2016,1,53,1,5


### Modifying the date and time property

In [27]:
train.timestamp = pd.to_datetime(train.timestamp, format='%Y-%m-%d %H:%M:%S')
train["Year"] = train.timestamp.dt.year
train["Month"] = train.timestamp.dt.month
train["Week"] = train.timestamp.dt.week
train["Day"] = train.timestamp.dt.day
train["hour"] = train.timestamp.dt.hour

C:\Users\piyus\AppData\Local\Temp\ipykernel_1920\349208782.py:4: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  train["Week"] = train.timestamp.dt.week


In [31]:
train.T

,6485009,6485010,6485011,6485012,6485013,6485014,6485016,6485017,6485018,6485019,...,20212937,20212938,20212940,20212941,20212942,20212951,20212952,20212955,20212956,20212963
building_id,565,565,565,565,565,565,565,565,565,565,...,591,591,591,591,591,591,591,591,591,591
meter,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
timestamp,2016-01-01 01:00:00,2016-01-01 02:00:00,2016-01-01 03:00:00,2016-01-01 04:00:00,2016-01-01 05:00:00,2016-01-01 06:00:00,2016-01-01 08:00:00,2016-01-01 09:00:00,2016-01-01 10:00:00,2016-01-01 11:00:00,...,2016-12-30 21:00:00,2016-12-30 22:00:00,2016-12-31 00:00:00,2016-12-31 01:00:00,2016-12-31 02:00:00,2016-12-31 11:00:00,2016-12-31 12:00:00,2016-12-31 15:00:00,2016-12-31 16:00:00,2016-12-31 23:00:00
meter_reading,2.4905,2.12425,2.1975,2.344,2.051,2.4905,2.051,2.344,2.051,2.12425,...,4.468631,4.175221,3.662119,3.807769,3.810787,4.468279,4.468602,4.540592,4.103055,3.443512
site_id,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
primary_use,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
square_feet,15326,15326,15326,15326,15326,15326,15326,15326,15326,15326,...,59326,59326,59326,59326,59326,59326,59326,59326,59326,59326
year_built,1954.0,1954.0,1954.0,1954.0,1954.0,1954.0,1954.0,1954.0,1954.0,1954.0,...,1932.0,1932.0,1932.0,1932.0,1932.0,1932.0,1932.0,1932.0,1932.0,1932.0
floor_count,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
air_temperature,9.4,8.3,7.8,5.0,5.6,5.0,4.4,4.4,4.4,4.4,...,12.8,14.4,12.2,11.7,11.1,10.0,9.4,7.2,7.2,12.8


In [32]:
# Note that 1. building_id, timestamp and site_id do not have any direct impact so we'll remove them
train.drop(columns=["building_id", "timestamp", "site_id"], inplace=True)

In [33]:
train.T

,6485009,6485010,6485011,6485012,6485013,6485014,6485016,6485017,6485018,6485019,...,20212937,20212938,20212940,20212941,20212942,20212951,20212952,20212955,20212956,20212963
meter,0.0000,0.00000,0.0000,0.000,0.000,0.0000,0.000,0.000,0.000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
meter_reading,2.4905,2.12425,2.1975,2.344,2.051,2.4905,2.051,2.344,2.051,2.12425,...,4.468631,4.175221,3.662119,3.807769,3.810787,4.468279,4.468602,4.540592,4.103055,3.443512
primary_use,0.0000,0.00000,0.0000,0.000,0.000,0.0000,0.000,0.000,0.000,0.00000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
square_feet,15326.0000,15326.00000,15326.0000,15326.000,15326.000,15326.0000,15326.000,15326.000,15326.000,15326.00000,...,59326.000000,59326.000000,59326.000000,59326.000000,59326.000000,59326.000000,59326.000000,59326.000000,59326.000000,59326.000000
year_built,1954.0000,1954.00000,1954.0000,1954.000,1954.000,1954.0000,1954.000,1954.000,1954.000,1954.00000,...,1932.000000,1932.000000,1932.000000,1932.000000,1932.000000,1932.000000,1932.000000,1932.000000,1932.000000,1932.000000
floor_count,2.0000,2.00000,2.0000,2.000,2.000,2.0000,2.000,2.000,2.000,2.00000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
air_temperature,9.4000,8.30000,7.8000,5.000,5.600,5.0000,4.400,4.400,4.400,4.40000,...,12.800000,14.400000,12.200000,11.700000,11.100000,10.000000,9.400000,7.200000,7.200000,12.800000
cloud_coverage,0.0000,0.00000,0.0000,0.000,0.000,0.0000,0.000,0.000,0.000,0.00000,...,4.000000,2.000000,2.000000,2.000000,4.000000,4.000000,2.000000,4.000000,2.000000,2.000000
dew_temperature,-2.2000,-2.80000,-2.8000,-0.600,-1.100,-1.1000,-2.200,-4.400,-6.700,-7.80000,...,7.200000,5.000000,6.100000,8.900000,8.900000,3.300000,3.300000,4.400000,3.900000,3.300000
precip_depth_1_hr,0.0000,0.00000,0.0000,0.000,0.000,0.0000,0.000,0.000,0.000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,0.000000


## Let us now train a model

In [34]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [36]:
X_train, X_test, y_train, y_test = train_test_split(train.drop("meter_reading", axis=1), train.meter_reading)

In [37]:
clf = RandomForestRegressor(n_jobs=-1)
clf.fit(X_train, y_train)

RandomForestRegressor(n_jobs=-1)

In [38]:
clf.score(X_test, y_test)

0.9954514977732327

In [39]:
y_preds = clf.predict(X_test)

In [40]:
from sklearn.metrics import mean_squared_log_error

In [41]:
mean_squared_log_error(y_test, y_preds)

0.017352983943522776

In [50]:
test.dtypes

row_id                         int64
building_id                    int64
meter                          int64
timestamp             datetime64[ns]
site_id                        int64
primary_use                     int8
square_feet                    int64
year_built                   float64
floor_count                  float64
air_temperature              float64
cloud_coverage               float64
dew_temperature              float64
precip_depth_1_hr            float64
sea_level_pressure           float64
wind_direction               float64
wind_speed                   float64
Year                           int64
Month                          int64
Week                           int64
Day                            int64
hour                           int64
dtype: object

In [47]:
train.dtypes

meter                   int64
meter_reading         float64
primary_use              int8
square_feet             int64
year_built            float64
floor_count           float64
air_temperature       float64
cloud_coverage        float64
dew_temperature       float64
precip_depth_1_hr     float64
sea_level_pressure    float64
wind_direction        float64
wind_speed            float64
Year                    int64
Month                   int64
Week                    int64
Day                     int64
hour                    int64
dtype: object

### enrich test data sets with date and time columns

In [45]:
test["Year"] = test.timestamp.dt.year
test["Month"] = test.timestamp.dt.month
test["Week"] = test.timestamp.dt.week
test["Day"] = test.timestamp.dt.day
test["hour"] = test.timestamp.dt.hour

C:\Users\piyus\AppData\Local\Temp\ipykernel_1920\1954988026.py:3: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  test["Week"] = test.timestamp.dt.week


In [48]:
test.shape, train.shape

((41697600, 21), (321728, 18))

## Convert primary use of test from object dtype to categorical

In [49]:
test.primary_use = test.primary_use.astype("category").cat.codes

In [52]:
# drop timestamp, site_id and building_id
test.drop(columns=["building_id", "timestamp", "site_id"], inplace=True)

In [57]:
test

,meter,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,Year,Month,Week,Day,hour
0,0,0,7432,2008.0,3.0,17.8,4.0,11.7,0.0,1021.4,100.0,3.6,2017,1,52,1,0
17520,0,0,2720,2004.0,3.0,17.8,4.0,11.7,0.0,1021.4,100.0,3.6,2017,1,52,1,0
35040,0,0,5376,1991.0,3.0,17.8,4.0,11.7,0.0,1021.4,100.0,3.6,2017,1,52,1,0
52560,0,0,23685,2002.0,3.0,17.8,4.0,11.7,0.0,1021.4,100.0,3.6,2017,1,52,1,0
70080,0,0,116607,1975.0,3.0,17.8,4.0,11.7,0.0,1021.4,100.0,3.6,2017,1,52,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41627519,0,1,19619,1914.0,3.0,13.3,0.0,7.2,-1.0,1020.8,70.0,2.6,2018,5,19,9,7
41645039,0,0,4298,1969.0,3.0,13.3,0.0,7.2,-1.0,1020.8,70.0,2.6,2018,5,19,9,7
41662559,0,1,11265,1997.0,3.0,13.3,0.0,7.2,-1.0,1020.8,70.0,2.6,2018,5,19,9,7
41680079,0,4,29775,2001.0,3.0,13.3,0.0,7.2,-1.0,1020.8,70.0,2.6,2018,5,19,9,7


In [54]:
# Sort the test data w.r.t row_id
test.sort_values(inplace=True, by=['row_id'])

In [56]:
# Now drop the row_id for training
test.drop(columns='row_id', inplace=True)

In [61]:
test.shape, X_test.shape

((41697600, 17), (80432, 17))

In [65]:
test_preds = clf.predict(test)

MemoryError: Unable to allocate 5.28 GiB for an array with shape (17, 41697600) and data type float64

In [ ]:
ans = mean_squared_log_error(test, test_preds)
ans

In [63]:
train = 1
building_data = 1
weather_test = 1
weather_train = 1

In [66]:
clf.predict(test).to_csv("hi.csv")

MemoryError: Unable to allocate 318. MiB for an array with shape (41697600, 1, 1) and data type float64